In [2]:
from sqlalchemy import create_engine
import sqlite3
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
import json
# Ruta al archivo
file_path = "/content/drive/MyDrive/Facultad/PFC/Implementacion/config.json"

# Leer el archivo
with open(file_path, 'r') as file:
    config = json.load(file)

###API KEY

In [4]:
import os

# os.environ["SEC_API_USER_AGENT"] =

os.environ["BEA_API_KEY"] = config["BEA_API_KEY"]

# os.environ["FRED_API_KEY"] =

###Install


In [5]:
!pip install finagg
!finagg install --help

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.8/149.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.0 MB/s eta 0:00:00
Usage: finagg install [OPTIONS]

  Set API keys/user agents, drop and recreate
  tables, and install the recommended datasets
  into the SQL database.

Options:
  -s, --skip [bea|fred|indices|sec|yfinance|fundam]
                                  Subpackage
                                  installations to
                                  skip. Useful to
                                  avoid
                                  reinstalling
                                  data that you
                                  don't want to
                                  when using the
                                  general install
                                  com

###Inspect Tables

In [6]:
from sqlalchemy import create_engine, inspect, MetaData

db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# Combine all metadata
all_metadata = MetaData()
# Setup the database and create tables based on the combined metadata
engine = create_engine(f'sqlite:///{db_path}')
all_metadata.create_all(engine)

# Reflect the tables from the engine into the metadata
all_metadata.reflect(bind=engine)

# Get a list of all the tables
tables = all_metadata.tables.keys()

# Use an inspector to fetch columns and other details
inspector = inspect(engine)

#Ver datos
for table in tables:
    print("Table:", table)
    for column in inspector.get_columns(table):
        print("  Column:", column['name'], column['type'])

Table: fixed_assets
  Column: table_id VARCHAR
  Column: series_code VARCHAR
  Column: line INTEGER
  Column: line_description VARCHAR
  Column: year INTEGER
  Column: metric VARCHAR
  Column: units VARCHAR
  Column: e INTEGER
  Column: value FLOAT
Table: fred.raw.series
  Column: series_id VARCHAR
  Column: realtime_start VARCHAR
  Column: realtime_end VARCHAR
  Column: date VARCHAR
  Column: value FLOAT
Table: fred.refined.economic
  Column: date VARCHAR
  Column: CIVPART FLOAT
  Column: LOG_CHANGE(CPIAUCNS) FLOAT
  Column: LOG_CHANGE(CSUSHPINSA) FLOAT
  Column: LOG_CHANGE(DJIA) FLOAT
  Column: FEDFUNDS FLOAT
  Column: LOG_CHANGE(GDP) FLOAT
  Column: LOG_CHANGE(GDPC1) FLOAT
  Column: GS10 FLOAT
  Column: LOG_CHANGE(M2) FLOAT
  Column: MICH FLOAT
  Column: LOG_CHANGE(NASDAQ100) FLOAT
  Column: LOG_CHANGE(NASDAQCOM) FLOAT
  Column: PSAVERT FLOAT
  Column: LOG_CHANGE(SP500) FLOAT
  Column: UMCSENT FLOAT
  Column: UNRATE FLOAT
  Column: LOG_CHANGE(WALCL) FLOAT
Table: fundam.refined.funda

###Number of Rows

In [7]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    table_name = table[0]
    # Colocar el nombre de la tabla entre comillas dobles
    cursor.execute(f'SELECT COUNT(*) FROM "{table_name}";')
    count = cursor.fetchone()[0]
    print(f"Table {table_name} has {count} rows.")

Table fred.raw.series has 34527 rows.
Table fred.refined.economic has 2353 rows.
Table indices.raw.djia has 30 rows.
Table indices.raw.sp500 has 503 rows.
Table indices.raw.nasdaq100 has 101 rows.
Table sec.raw.submissions has 471 rows.
Table sec.raw.tags has 172005 rows.
Table sec.refined.annual has 1276 rows.
Table sec.refined.annual.normalized has 664 rows.
Table sec.refined.quarterly has 3519 rows.
Table sec.refined.quarterly.normalized has 3113 rows.
Table yfinance.raw.prices has 4257432 rows.
Table yfinance.refined.daily has 4039449 rows.
Table fundam.refined.fundam has 283344 rows.
Table fundam.refined.fundam.normalized has 263607 rows.
Table fixed_assets has 542689 rows.
Table gdp_by_industry has 151194 rows.
Table input_output has 587346 rows.
Table nipa has 0 rows.


###Tickers

In [8]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT ticker FROM 'fundam.refined.fundam'")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('A',)
('AAPL',)
('ALGN',)
('ALLE',)
('AMAT',)
('AMCR',)
('ANET',)
('APTV',)
('ATVI',)
('AVGO',)
('AXON',)
('CARR',)
('CDW',)
('CLX',)
('COO',)
('COST',)
('CPRT',)
('CRL',)
('CSCO',)
('CTLT',)
('CVS',)
('CZR',)
('D',)
('DD',)
('DVA',)
('DXCM',)
('EA',)
('ECL',)
('EIX',)
('ENPH',)
('EW',)
('FANG',)
('FE',)
('FSLR',)
('FTNT',)
('GE',)
('GEHC',)
('GM',)
('GNRC',)
('GOOG',)
('GPN',)
('HCA',)
('HD',)
('HII',)
('HLT',)
('HPE',)
('HSIC',)
('HWM',)
('IDXX',)
('INCY',)
('ISRG',)
('JNPR',)
('KDP',)
('KMI',)
('KVUE',)
('LCID',)
('LDOS',)
('LH',)
('LHX',)
('LOW',)
('LRCX',)
('LULU',)
('LVS',)
('LW',)
('MAS',)
('MDT',)
('MELI',)
('MGM',)
('MHK',)
('MLM',)
('MMM',)
('MPC',)
('MPWR',)
('MRNA',)
('MRO',)
('MSFT',)
('MTD',)
('MU',)
('NCLH',)
('NEE',)
('NOC',)
('NTAP',)
('ON',)
('OTIS',)
('PAYC',)
('PNR',)
('PODD',)
('POOL',)
('PPG',)
('PWR',)
('QCOM',)
('QRVO',)
('REGN',)
('RMD',)
('ROL',)
('RVTY',)
('SBUX',)
('SEE',)
('SIRI',)
('SJM',)
('SLB',)
('SNPS',)
('SPGI',)
('STE',)
('STLD',)
('STX',)
('SWKS',)

###Data Range

In [9]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# Combine all metadata
# Setup the database and create tables based on the combined metadata
engine = create_engine(f'sqlite:///{db_path}')

def get_date_range_from_table(engine, table_name, column_name, value):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(date), MAX(date)
            FROM "{table_name}"
            WHERE {column_name} = :value
        """)
        result = connection.execute(query, {"value": value}).fetchone()
        return result if result else (None, None)


def get_SEC_date_range_from_table(engine, table_name, column_name, value):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(fy), MAX(fy), MIN(fp), MAX(fp)
            FROM "{table_name}"
            WHERE {column_name} = :value
        """)
        result = connection.execute(query, {"value": value}).fetchone()
        if result:
            min_year, max_year, min_period, max_period = result
            start_date = f"{min_year}-{min_period}"
            end_date = f"{max_year}-{max_period}"
            return start_date, end_date
        else:
            return None, None

def get_fred_range(engine):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(date), MAX(date)
            FROM "fred.refined.economic"
        """)
        result = connection.execute(query).fetchone()
        return result if result else (None, None)


def get_bea_range(engine,table_name):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(year), MAX(year)
            FROM "{table_name}"
        """)
        result = connection.execute(query).fetchone()
        return result if result else (None, None)

In [10]:
from sqlalchemy import text

ticker = "AAPL"  # Coloca el ticker que quieres consultar aquí
ticker_tables = ["yfinance.refined.daily", "fundam.refined.fundam", "fundam.refined.fundam.normalized"]  # tablas

cik_query = text("""
  SELECT cik
  FROM "sec.raw.submissions"
  WHERE ticker = :ticker
  LIMIT 1
""")
with engine.connect() as connection:
    cik = connection.execute(cik_query, {'ticker': ticker}).scalar()

cik_tables = ["sec.refined.quarterly.normalized"]  # Agrega las demás tablas que correspondan a 'cik'

#Macroeconomic data
start_date, end_date = get_fred_range(engine)
print(f"Table fred.refined.economic: Data range: {start_date} a {end_date}")

print("Datos BEA:")
start_date, end_date = get_bea_range(engine,"fixed_assets")
print(f"Table fixed_assets: Data range: {start_date} to {end_date}")

start_date, end_date = get_bea_range(engine,"gdp_by_industry")
print(f"Table gdp_by_industry: Data range: {start_date} to {end_date}")

start_date, end_date = get_bea_range(engine,"input_output")
print(f"Table input_output: Data range: {start_date} to {end_date}")


# Obtener rangos de fechas para tablas basadas en ticker
print(f"Empresa: {ticker}")
for table in ticker_tables:
  start_date, end_date = get_date_range_from_table(engine, table, "ticker", ticker)
  print(f"Table {table}: Data range: {start_date} to {end_date}")

# Obtener rangos de fechas para tablas basadas en cik
start_date, end_date = get_SEC_date_range_from_table(engine, "sec.refined.quarterly.normalized", "cik", cik)
print(f"Table sec.refined.quarterly.normalized: Data range: {start_date} to {end_date}")


Table fred.refined.economic: Data range: 2014-10-02 a 2023-08-28
Datos BEA:
Table fixed_assets: Data range: 1901 to 2021
Table gdp_by_industry: Data range: 2005 to 2023
Table input_output: Data range: 1997 to 2021
Empresa: AAPL
Table yfinance.refined.daily: Data range: 1980-12-15 to 2023-08-29
Table fundam.refined.fundam: Data range: 2010-01-25 to 2023-08-29
Table fundam.refined.fundam.normalized: Data range: 2010-01-25 to 2023-08-29
Table sec.refined.quarterly.normalized: Data range: 2010-Q1 to 2023-Q3


###Data SEC for Specific Ticker

In [11]:
# def get_data_sec_ticker(engine, ticker):
#     cik_query = text("""
#       SELECT cik
#       FROM "sec.raw.submissions"
#       WHERE ticker = :ticker
#       LIMIT 1
#     """)
#     with engine.connect() as connection:
#         value_cik = connection.execute(cik_query, {'ticker': ticker}).scalar()

#     with engine.connect() as connection:
#         query = text(f"""
#             SELECT *
#             FROM "sec.refined.quarterly.normalized"
#             WHERE cik = :value
#         """)
#         result = connection.execute(query, {"value": value_cik}).fetchall()
#         return result if result else None

# ticker = "AAPL"
# data = get_data_sec_ticker(engine,ticker)
# for d in data:
#   print(d)

In [12]:
import sqlite3
import pandas as pd

def get_data_sec_ticker(db_path, ticker):
    connection = sqlite3.connect(db_path)
    cik_query = """
      SELECT cik
      FROM "sec.raw.submissions"
      WHERE ticker = ?
      LIMIT 1
    """
    value_cik = connection.execute(cik_query, (ticker,)).fetchone()[0]

    query = f"""
        SELECT *
        FROM "sec.raw.tags"
        WHERE cik = ?
    """
    data = pd.read_sql(query, connection, params=(value_cik,))
    connection.close()

    return data

ticker = "AAPL"
data = get_data_sec_ticker(db_path, ticker)
data


,cik,accn,taxonomy,tag,form,units,fy,fp,start,end,filed,frame,label,description,entity,value
0,0000320193,0001193125-09-214859,us-gaap,Assets,10-K,USD,2009,FY,None,2008-09-27,2009-10-27,None,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,3.957200e+10
1,0000320193,0001193125-09-153165,us-gaap,Assets,10-Q,USD,2009,Q3,None,2008-09-27,2009-07-22,CY2009Q2I,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,3.957200e+10
2,0000320193,0001193125-10-238044,us-gaap,Assets,10-K,USD,2010,FY,None,2008-09-27,2010-10-27,CY2008Q3I,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,3.617100e+10
3,0000320193,0001193125-10-012085,us-gaap,Assets,10-Q,USD,2010,Q1,None,2009-09-26,2010-01-25,CY2009Q4I,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,4.750100e+10
4,0000320193,0001193125-10-088957,us-gaap,Assets,10-Q,USD,2010,Q2,None,2009-09-26,2010-04-21,CY2010Q1I,Assets,Sum of the carrying amounts as of the balance ...,Apple Inc.,4.750100e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,0000320193,0000320193-22-000059,us-gaap,StockholdersEquity,10-Q,USD,2022,Q2,None,2020-09-26,2022-04-29,CY2020Q4I,Stockholders' Equity Attributable to Parent,Total of all stockholders' equity (deficit) it...,Apple Inc.,6.533900e+10
509,0000320193,0000320193-22-000070,us-gaap,StockholdersEquity,10-Q,USD,2022,Q3,None,2020-09-26,2022-07-29,CY2021Q1I,Stockholders' Equity Attributable to Parent,Total of all stockholders' equity (deficit) it...,Apple Inc.,6.533900e+10
510,0000320193,0000320193-23-000006,us-gaap,StockholdersEquity,10-Q,USD,2023,Q1,None,2021-09-25,2023-02-03,None,Stockholders' Equity Attributable to Parent,Total of all stockholders' equity (deficit) it...,Apple Inc.,6.309000e+10
511,0000320193,0000320193-23-000064,us-gaap,StockholdersEquity,10-Q,USD,2023,Q2,None,2021-09-25,2023-05-05,CY2021Q4I,Stockholders' Equity Attributable to Parent,Total of all stockholders' equity (deficit) it...,Apple Inc.,6.309000e+10


###Show Prices

In [13]:
def get_data_price(db_path, ticker):
    connection = sqlite3.connect(db_path)
    query = f"""
        SELECT *
        FROM "yfinance.refined.daily"
        WHERE ticker = ?
    """
    data = pd.read_sql(query, connection, params=(ticker,))
    connection.close()

    return data

ticker = "AAPL"
data = get_data_price(db_path, ticker)
data

,ticker,date,LOG_CHANGE(open),LOG_CHANGE(high),LOG_CHANGE(low),LOG_CHANGE(close),LOG_CHANGE(volume),"LOG_CHANGE(high, open)","LOG_CHANGE(low, open)"
0,AAPL,1980-12-15,-0.049004,-0.053342,-0.053581,-0.053581,-0.980845,0.000000,-0.004576
1,AAPL,1980-12-16,-0.075869,-0.075869,-0.076231,-0.076231,-0.508959,0.000000,-0.004938
2,AAPL,1980-12-17,0.019511,0.024330,0.024449,0.024449,-0.201401,0.004819,0.000000
3,AAPL,1980-12-18,0.028580,0.028445,0.028580,0.028580,-0.162870,0.004684,0.000000
4,AAPL,1980-12-19,0.059239,0.058970,0.059239,0.059239,-0.412351,0.004415,0.000000
...,...,...,...,...,...,...,...,...,...
10762,AAPL,2023-08-23,0.008212,0.021547,0.011732,0.021711,0.225376,0.016830,-0.001065
10763,AAPL,2023-08-24,0.011971,-0.002482,-0.013095,-0.026519,0.041299,0.002377,-0.026131
10764,AAPL,2023-08-25,-0.018378,-0.010826,-0.001080,0.012564,-0.066345,0.009929,-0.008834
10765,AAPL,2023-08-28,0.015162,0.008006,0.015408,0.008807,-0.161060,0.002773,-0.008588


###Show Tags Type

In [14]:
import sqlite3
import pandas as pd

def get_tags(db_path):
    connection = sqlite3.connect(db_path)
    query = f"""
        SELECT DISTINCT tag
        FROM "sec.raw.tags"
    """
    data = pd.read_sql(query, connection)
    #data = pd.read_sql(query, connection)

    connection.close()

    return data

data = get_tags(db_path)
data

,tag
0,Assets
1,AssetsCurrent
2,EarningsPerShareBasic
3,InventoryNet
4,LiabilitiesCurrent
5,NetIncomeLoss
6,StockholdersEquity
7,CommonStockSharesOutstanding
8,Liabilities


###Show Tags by Ticker

In [15]:
def get_tags_ticker(db_path, ticker):
    #Get the company cik by ticker
    connection = sqlite3.connect(db_path)
    cik_query = """
      SELECT cik
      FROM "sec.raw.submissions"
      WHERE ticker = ?
      LIMIT 1
    """
    value_cik = connection.execute(cik_query, (ticker,)).fetchone()[0]

    query = f"""
        SELECT *
        FROM "sec.refined.quarterly"
        WHERE cik = ?
    """
    data = pd.read_sql(query, connection, params=(value_cik,))
    #data = pd.read_sql(query, connection)

    connection.close()

    return data

ticker = "MSFT"
data = get_tags_ticker(db_path, ticker)
data

,cik,filed,fy,fp,LOG_CHANGE(Assets),LOG_CHANGE(AssetsCurrent),LOG_CHANGE(CommonStockSharesOutstanding),LOG_CHANGE(InventoryNet),LOG_CHANGE(Liabilities),LOG_CHANGE(LiabilitiesCurrent),LOG_CHANGE(StockholdersEquity),AssetCoverageRatio,BookRatio,DebtEquityRatio,EarningsPerShareBasic,QuickRatio,ReturnOnAssets,ReturnOnEquity,WorkingCapitalRatio
0,0000789019,2011-01-27,2011,Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071871,1.501477,5.327065,0.901881,1.15,2.101044,0.118867,0.231150,2.129346
1,0000789019,2011-04-28,2011,Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031716,1.501477,5.327065,0.873726,1.61,2.101044,0.165387,0.311573,2.129346
2,0000789019,2011-10-20,2012,Q1,0.232968,0.296845,-0.034268,0.617375,0.256600,0.095738,0.026596,1.548401,6.815067,1.099676,0.63,2.555988,0.049768,0.115249,2.603670
3,0000789019,2012-04-19,2012,Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.129871,1.548401,6.815067,0.965745,2.03,2.555988,0.158927,0.323206,2.603670
4,0000789019,2012-10-18,2013,Q1,0.109399,0.127245,0.000597,-0.187876,0.061730,0.127536,0.105359,1.613299,7.918268,0.924517,0.68,2.568129,0.047316,0.096614,2.602912
5,0000789019,2013-01-24,2013,Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076629,1.613299,7.918268,0.856319,1.47,2.568129,0.101937,0.192792,2.602912
6,0000789019,2013-04-18,2013,Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068380,1.613299,7.918268,0.799720,2.08,2.568129,0.144058,0.254446,2.602912
7,0000789019,2013-10-24,2014,Q1,0.160830,0.176085,-0.006344,0.533263,0.145176,0.135117,0.002575,1.654102,9.479347,0.922294,0.53,2.659967,0.031356,0.064879,2.711762
8,0000789019,2014-01-23,2014,Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052907,1.654102,9.479347,0.874766,1.29,2.659967,0.076128,0.149402,2.711762
9,0000789019,2014-04-24,2014,Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055111,1.654102,9.479347,0.827861,2.02,2.659967,0.118640,0.220347,2.711762


###BEA Data

In [16]:
import finagg
import pandas as pd
from sqlalchemy import text

def show_bea_data(engine,table):
  with engine.begin() as conn:
      query = text(f"""SELECT * FROM {table}""")
      df = pd.read_sql_query(query, conn)
  return df

####Input_Output

In [17]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
engine = create_engine(f'sqlite:///{db_path}')

with engine.connect() as connection:
    result = connection.execute(text("SELECT sql FROM sqlite_master WHERE name = 'input_output';")) #Datos de la table
    for row in result:
        print(row)

('CREATE TABLE input_output (\n\ttable_id INTEGER NOT NULL, \n\tyear INTEGER NOT NULL, \n\trow_code VARCHAR NOT NULL, \n\trow_description VARCHAR, \n\t ... (23 characters truncated) ... ol_code VARCHAR NOT NULL, \n\tcol_description VARCHAR, \n\tcol_type VARCHAR, \n\tvalue FLOAT, \n\tPRIMARY KEY (table_id, year, row_code, col_code)\n)',)


In [18]:
df_in_out=show_bea_data(engine,"input_output")
df_in_out

,table_id,year,row_code,row_description,row_type,col_code,col_description,col_type,value
0,6000,1997,,Total commodity output,Commodity,311FT,Food and beverage and tobacco products,Commodity,2.673929e+00
1,6000,1997,,Total commodity output,Commodity,481,Air transportation,Commodity,1.951930e+00
2,6000,1997,,Total commodity output,Commodity,487OS,Other transportation and support activities,Commodity,1.653248e+00
3,6000,1997,,Total commodity output,Commodity,22,Utilities,Commodity,1.700775e+00
4,6000,1997,,Total commodity output,Commodity,493,Warehousing and storage,Commodity,1.354362e+00
...,...,...,...,...,...,...,...,...,...
587341,6010,2021,VAPRO,Value Added (producer prices),Commodity,22,Utilities,Industry,3.784360e+05
587342,6010,2021,VAPRO,Value Added (producer prices),Commodity,81,"Other services, except government",Industry,4.698510e+05
587343,6010,2021,VAPRO,Value Added (producer prices),Commodity,21,Mining,Industry,3.339350e+05
587344,6010,2021,VAPRO,Value Added (producer prices),Commodity,11,"Agriculture, forestry, fishing, and hunting",Industry,2.065680e+05


######Tipos de tablas con su descripcion

In [19]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT DISTINCT row_code FROM input_output")) #Datos de la table
    for row in result:
        print(row)

with engine.connect() as connection:
    result = connection.execute(text("SELECT DISTINCT col_code FROM input_output")) #Datos de la table
    for row in result:
        print(row)

('',)
('111CA',)
('113FF',)
('211',)
('212',)
('213',)
('22',)
('23',)
('311FT',)
('313TT',)
('315AL',)
('321',)
('322',)
('323',)
('324',)
('325',)
('326',)
('327',)
('331',)
('332',)
('333',)
('334',)
('335',)
('3361MV',)
('3364OT',)
('337',)
('339',)
('42',)
('441',)
('445',)
('452',)
('481',)
('482',)
('483',)
('484',)
('485',)
('486',)
('487OS',)
('493',)
('4A0',)
('511',)
('512',)
('513',)
('514',)
('521CI',)
('523',)
('524',)
('525',)
('532RL',)
('5411',)
('5412OP',)
('5415',)
('55',)
('561',)
('562',)
('61',)
('621',)
('622',)
('623',)
('624',)
('711AS',)
('713',)
('721',)
('722',)
('81',)
('GFE',)
('GFGD',)
('GFGN',)
('GSLE',)
('GSLG',)
('HS',)
('ORE',)
('Other',)
('Used',)
('11',)
('21',)
('31G',)
('44RT',)
('48TW',)
('51',)
('6',)
('7',)
('FIRE',)
('G',)
('PROF',)
('T017',)
('T005',)
('T00OTOP',)
('T00SUB',)
('T00TOP',)
('T018',)
('V001',)
('V003',)
('VABAS',)
('VAPRO',)
('T00OSUB',)
('111CA',)
('113FF',)
('211',)
('212',)
('213',)
('22',)
('23',)
('311FT',)
('313TT',)
('315

In [33]:
table_id = 6000
while(table_id!=6011):
  with engine.connect() as connection:
      result = connection.execute(text(f"SELECT * FROM input_output WHERE table_id={table_id} LIMIT 5")) #Datos de la table
      for row in result:
          print(row)
      table_id+=1

(6000, 1997, '', 'Total commodity output', 'Commodity', '111CA', 'Farms', 'Commodity', 2.270805597305298)
(6000, 1997, '', 'Total commodity output', 'Commodity', '113FF', 'Forestry, fishing, and related activities', 'Commodity', 1.939224123954773)
(6000, 1997, '', 'Total commodity output', 'Commodity', '211', 'Oil and gas extraction', 'Commodity', 1.8392596244812012)
(6000, 1997, '', 'Total commodity output', 'Commodity', '212', 'Mining, except oil and gas', 'Commodity', 2.108003616333008)
(6000, 1997, '', 'Total commodity output', 'Commodity', '213', 'Support activities for mining', 'Commodity', 1.679688811302185)
(6001, 1997, '', 'Total commodity output', 'Commodity', '11', 'Agriculture, forestry, fishing, and hunting', 'Commodity', 2.1731319427490234)
(6001, 1997, '', 'Total commodity output', 'Commodity', '21', 'Mining', 'Commodity', 1.9003849029541016)
(6001, 1997, '', 'Total commodity output', 'Commodity', '22', 'Utilities', 'Commodity', 1.6299370527267456)
(6001, 1997, '', 'Tota

In [23]:
finagg.bea.api.input_output.get_parameter_values("TableID")  #El key no se condicen con los table_id. Voy a asumir que van en orden.

,Key,Desc
0,59,"Commodity-by-Commodity Total Requirements, Aft..."
1,58,"Commodity-by-Commodity Total Requirements, Aft..."
2,57,"Industry-by-Commodity Total Requirements, Afte..."
3,56,"Industry-by-Commodity Total Requirements, Afte..."
4,61,"Industry-by-Industry Total Requirements, After..."
5,60,"Industry-by-Industry Total Requirements, After..."
6,262,The Domestic Supply of Commodities by Industri...
7,261,The Domestic Supply of Commodities by Industri...
8,259,The Use of Commodities by Industries - Summary
9,258,The Use of Commodities by Industries - Sector


####Fixed_Assets

In [24]:
df_fix_as=show_bea_data(engine,"fixed_assets")
df_fix_as

,table_id,series_code,line,line_description,year,metric,units,e,value
0,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1925,Current Dollars,Level,9,297.500000
1,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1926,Current Dollars,Level,9,307.600006
2,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1927,Current Dollars,Level,9,315.500000
3,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1928,Current Dollars,Level,9,328.200012
4,FAAt101,k1wtotl1es00,1,Fixed assets and consumer durable goods,1929,Current Dollars,Level,9,333.600006
...,...,...,...,...,...,...,...,...,...
542684,FAAt903,ixgstlc1es00,24,State and local,2017,Chained Dollars,Level,9,358.700012
542685,FAAt903,ixgstlc1es00,24,State and local,2018,Chained Dollars,Level,9,367.200012
542686,FAAt903,ixgstlc1es00,24,State and local,2019,Chained Dollars,Level,9,372.799988
542687,FAAt903,ixgstlc1es00,24,State and local,2020,Chained Dollars,Level,9,375.500000


In [25]:
finagg.bea.api.fixed_assets.get_parameter_values("TableName")

,TableName,Description
0,FAAt101,Table 1.1. Current-Cost Net Stock of Fixed Ass...
1,FAAt102,Table 1.2. Chain-Type Quantity Indexes for Net...
2,FAAt103,Table 1.3. Current-Cost Depreciation of Fixed ...
3,FAAt104,Table 1.4. Chain-Type Quantity Indexes for Dep...
4,FAAt105,Table 1.5. Investment in Fixed Assets and Cons...
...,...,...
104,FAAt809,Table 8.9. Current-Cost Average Age at Yearend...
105,FAAt810,Table 8.10. Historical-Cost Average Age at Yea...
106,FAAt901,Table 9.1. Real Net Stock of Fixed Assets and ...
107,FAAt902,Table 9.2. Real Depreciation of Fixed Assets a...


In [71]:
with engine.connect() as connection:
    query = text("SELECT * FROM fixed_assets WHERE table_id = 'FAAt102'")
    df=pd.read_sql(query,connection)
df


,table_id,series_code,line,line_description,year,metric,units,e,value
0,FAAt102,kcwtotl1es00,1,Fixed assets and consumer durable goods,1925,Fisher Quantity Index,Level,0,9.025000
1,FAAt102,kcwtotl1es00,1,Fixed assets and consumer durable goods,1926,Fisher Quantity Index,Level,0,9.388000
2,FAAt102,kcwtotl1es00,1,Fixed assets and consumer durable goods,1927,Fisher Quantity Index,Level,0,9.727000
3,FAAt102,kcwtotl1es00,1,Fixed assets and consumer durable goods,1928,Fisher Quantity Index,Level,0,10.049000
4,FAAt102,kcwtotl1es00,1,Fixed assets and consumer durable goods,1929,Fisher Quantity Index,Level,0,10.355000
...,...,...,...,...,...,...,...,...,...
2323,FAAt102,kcgstlc1es00,24,State and local,2017,Fisher Quantity Index,Level,0,105.292000
2324,FAAt102,kcgstlc1es00,24,State and local,2018,Fisher Quantity Index,Level,0,106.523003
2325,FAAt102,kcgstlc1es00,24,State and local,2019,Fisher Quantity Index,Level,0,107.765999
2326,FAAt102,kcgstlc1es00,24,State and local,2020,Fisher Quantity Index,Level,0,108.987000


####GDP

In [26]:
df_gdp=show_bea_data(engine,"gdp_by_industry")
df_gdp

,table_id,freq,year,quarter,industry,industry_description,value
0,1,Q,2005,1,11,"Agriculture, forestry, fishing, and hunting",131.000000
1,1,Q,2005,1,111CA,Farms,107.500000
2,1,Q,2005,1,113FF,"Forestry, fishing, and related activities",23.500000
3,1,Q,2005,1,21,Mining,196.399994
4,1,Q,2005,1,211,Oil and gas extraction,133.000000
...,...,...,...,...,...,...,...
151189,209,Q,2023,1,ORE,Other real estate,824.000000
151190,209,Q,2023,1,PGOOD,Private goods-producing industries<sup>2</sup>,4942.899902
151191,209,Q,2023,1,PROF,Professional and business services,1770.800049
151192,209,Q,2023,1,PSERV,Private services-producing industries<sup>3</sup>,9821.299805


In [27]:
finagg.bea.api.gdp_by_industry.get_parameter_list()

,ParameterName,ParameterDataType,ParameterDescription,ParameterIsRequiredFlag,ParameterDefaultValue,MultipleAcceptedFlag,AllValue
0,Frequency,string,"A - Annual, Q-Quarterly",1,,1,ALL
1,Industry,string,List of industries to retrieve (ALL for All),1,,1,ALL
2,TableID,integer,The unique GDP by Industry table identifier (A...,1,,1,ALL
3,Year,integer,List of year(s) of data to retrieve (ALL for All),1,,1,ALL


In [28]:
finagg.bea.api.gdp_by_industry.get_parameter_values("Industry") #Params: "Frequency", "Industry", "TableID","Year"

,Key,Desc
0,11,"Agriculture, forestry, fishing, and hunting (A,Q)"
1,111CA,"Farms (A,Q)"
2,113FF,"Forestry, fishing, and related activities (A,Q)"
3,21,"Mining (A,Q)"
4,211,"Oil and gas extraction (A,Q)"
...,...,...
99,ORE,"Other real estate (A,Q)"
100,PGOOD,"Private goods-producing industries (A,Q)"
101,PROF,"Professional and business services (A,Q)"
102,PSERV,"Private services-producing industries (A,Q)"


In [29]:
###Borrar columnas: Index

In [30]:
import finagg
finagg.bea.api.get_dataset_list()

,DatasetName,DatasetDescription
0,NIPA,Standard NIPA tables
1,NIUnderlyingDetail,Standard NI underlying detail tables
2,MNE,Multinational Enterprises
3,FixedAssets,Standard Fixed Assets tables
4,ITA,International Transactions Accounts
5,IIP,International Investment Position
6,InputOutput,Input-Output Data
7,IntlServTrade,International Services Trade
8,GDPbyIndustry,GDP by Industry
9,Regional,Regional data sets


In [31]:
finagg.bea.api.fixed_assets.get_parameter_list()

,ParameterName,ParameterDataType,ParameterDescription,ParameterIsRequiredFlag,ParameterDefaultValue,MultipleAcceptedFlag,AllValue
0,TableName,string,The new Fixed Assets identifier,1,,0,
1,Year,integer,List of year(s) of data to retrieve (X for All),1,,1,X


In [32]:
finagg.bea.api.fixed_assets.get_parameter_values("TableName")

,TableName,Description
0,FAAt101,Table 1.1. Current-Cost Net Stock of Fixed Ass...
1,FAAt102,Table 1.2. Chain-Type Quantity Indexes for Net...
2,FAAt103,Table 1.3. Current-Cost Depreciation of Fixed ...
3,FAAt104,Table 1.4. Chain-Type Quantity Indexes for Dep...
4,FAAt105,Table 1.5. Investment in Fixed Assets and Cons...
...,...,...
104,FAAt809,Table 8.9. Current-Cost Average Age at Yearend...
105,FAAt810,Table 8.10. Historical-Cost Average Age at Yea...
106,FAAt901,Table 9.1. Real Net Stock of Fixed Assets and ...
107,FAAt902,Table 9.2. Real Depreciation of Fixed Assets a...
